In [147]:
%load_ext autoreload
%autoreload 2
import csv
import numpy as np
import pandas as pd
import math
import altair as alt
alt.data_transformers.enable("vegafusion")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DataTransformerRegistry.enable('vegafusion')

In [120]:
COLUMN_COUNT = 50
PAGE_SIZE = 4096

FILE_NAME = "/product/app/YouTube/YouTube.apk"
ZIP_ENTRY_NAME = 'classes.dex'

In [37]:
mapped_faults = []
with open("mapped_faults.csv") as csv_file:
    for row in csv.DictReader(csv_file):
        row['offset'] = int(row['offset'])
        mapped_faults.append(row)

file_sizes = []
with open("file_sizes.csv") as csv_file:
    for row in csv.DictReader(csv_file):
        row['size'] = int(row['size'])
        file_sizes.append(row)

NameError: name 'faults' is not defined

In [148]:

def page_fault_chart(file_name, zip_entry_name, file_sizes, mapped_faults):
    file =[file for file in file_sizes if file["file_name"] == file_name and file["zip_entry_name"] == zip_entry_name][0]
    file_size = file['size']
    file_offset = file['file_offset']
    
    faults = pd.DataFrame([
        entry
        for entry in mapped_faults
        if entry["file_name"] == file_name
        and entry["zip_entry_name"] == zip_entry_name
    ])

    rows = math.ceil(math.ceil(file_size / PAGE_SIZE) / COLUMN_COUNT)
    x, y = np.meshgrid(range(0, COLUMN_COUNT), range(0, rows))

    fault_offsets = faults["offset"].tolist()

    pages_accessed = set()
    
    for fault_offset in fault_offsets:
        offset = fault_offset - file_offset
        page = math.floor(offset / PAGE_SIZE)
        x_idx = page % COLUMN_COUNT
        y_idx = math.floor(page / COLUMN_COUNT)
        pages_accessed.add((x_idx,y_idx))
    
    z = np.array(x)
    for idx, _ in np.ndenumerate(x):
        z[idx] = 1 if idx in pages_accessed else 0
    
    # Convert this grid to columnar data expected by Altair
    source = pd.DataFrame({'x': x.ravel(),
                         'y': y.ravel(),
                         'z': z.ravel()})
    return alt.Chart(source).mark_rect().transform_calculate(
        offset='(datum.x + datum.y * 50) * 4096',
    ).encode(
        x='x:O',
        y='y:O',
        color='z:N',
        tooltip=['offset:Q'],
    ).properties(title=f"Page Faults for {file_name} - {zip_entry_name}")

charts = []
for dex_file in [file["zip_entry_name"] for file in file_sizes if file["file_name"] == FILE_NAME and file["zip_entry_name"] and file["zip_entry_name"].endswith(".dex")]:
    charts.append(page_fault_chart(FILE_NAME, dex_file,file_sizes, mapped_faults))

alt.vconcat(*charts).display()

alt.VConcatChart(...)

In [155]:

faults = pd.DataFrame([
    entry
    for entry in mapped_faults
    if entry["file_name"] == FILE_NAME
    and entry["zip_entry_name"] == "classes6.dex"
])
file_size =[file["size"] for file in file_sizes if file["file_name"] == file_name and file["zip_entry_name"] == zip_entry_name][0]
print(file_size)
print(faults)

page = math.floor(37796724 / PAGE_SIZE)
x_idx = page % COLUMN_COUNT
y_idx = math.floor(page / COLUMN_COUNT)
print(x_idx, y_idx)

7924512
                 process_name      thread_name  \
0  com.google.android.youtube  android.youtube   
1  com.google.android.youtube  android.youtube   
2  com.google.android.youtube  android.youtube   
3  com.google.android.youtube  android.youtube   
4  com.google.android.youtube  android.youtube   
5  com.google.android.youtube  android.youtube   
6  com.google.android.youtube  android.youtube   
7  com.google.android.youtube  android.youtube   

                          file_name zip_entry_name    offset  
0  /product/app/YouTube/YouTube.apk   classes6.dex  37796724  
1  /product/app/YouTube/YouTube.apk   classes6.dex  38189212  
2  /product/app/YouTube/YouTube.apk   classes6.dex  38125545  
3  /product/app/YouTube/YouTube.apk   classes6.dex  37872816  
4  /product/app/YouTube/YouTube.apk   classes6.dex  38005096  
5  /product/app/YouTube/YouTube.apk   classes6.dex  37827396  
6  /product/app/YouTube/YouTube.apk   classes6.dex  38033956  
7  /product/app/YouTube/YouTube.apk  